In [5]:
from data_service.data_master import DataMaster
import data_service.wrappers.eod_wrapper as wrap
import pandas as pd
import numpy as np
import asyncio
from alpha_tickers import alpha_tickers
from scipy import stats
pd.options.mode.chained_assignment = None
dm = DataMaster(config_file_path=r'C:\Users\thoma\quantdb\data_service\config.json')


[INFO]-setting up API-client for environment practice


In [24]:
exchanges = ['US' for _ in alpha_tickers]
columns = ['ticker','spike>1', 'spike>2', 'spike>3', 'spike>4', 
           'spike>5', 'wait_1_sd', 'wait_2_sd', 
           'wait_3_sd', 'wait_4_sd', 'wait_5_sd','ks_stat','ks_pval', 'shapiro_wilk_stat','shapiro_wilk_pval', 'anderson_darling_15%_pass', 'anderson_darling_10%_pass', 'anderson_darling_5%_pass', 'anderson_darling_2.5%_pass','anderson_darling_1%_pass']
stats_df = pd.DataFrame(columns=columns)
dfs = await wrap.new_asyn_batch_get_ohlcv(alpha_tickers, exchanges)
for df in dfs.values():
    df.set_index('datetime', inplace=True)

[DEBUG]-try https://eodhistoricaldata.com/api/eod/AMD.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/CVX.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/ALK.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/CHWY.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/AGNC.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/ADI.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/EXC.US?api_token=6318e406a42036.19146604&fmt=json&from=2014-02-03&to=2024-02-01 0
[DEBUG]-try https://eodhistoricaldata.com/api/eod/DKNG.US?api_token=6318e4

In [35]:
TICKER = 'RTX'
DATE_FROM = '2023-01-01'
DATE_TO = '2024-01-29'
DAY_MULTIPLE = 1
ROLLING_WINDOW = 20
LOOP_TICKERS = False

if LOOP_TICKERS:
    
    for ticker in dfs:
    
        select_data = dfs[ticker]
        slice_data = select_data.loc[DATE_FROM:DATE_TO]
        slice_data.loc[:, 'lagged_close'] = slice_data['close'].shift(1)
        slice_data.loc[:, 'prior_lagged_close'] = slice_data['lagged_close'].shift(DAY_MULTIPLE)
        slice_data.loc[:, 'simple_ret_cc'] = slice_data['lagged_close'] / slice_data['prior_lagged_close']
        slice_data.loc[:, 'abs_ret_cc'] = slice_data['close'] - slice_data['close'].shift(DAY_MULTIPLE)
        slice_data.loc[:, 'log_ret_cc'] = np.log(slice_data['simple_ret_cc'])
        slice_data.loc[:, 'n_period_vol_cc'] = slice_data['log_ret_cc'].rolling(ROLLING_WINDOW).std()  # n period volatility
        slice_data.loc[:, 'annualised_vol_cc'] = slice_data['n_period_vol_cc'] * np.sqrt(DAY_MULTIPLE)
        slice_data.loc[:, 'n_period_vol_price_terms'] = slice_data['n_period_vol_cc'] * slice_data['lagged_close']
        slice_data.loc[:, 'n_period_spike'] = slice_data['abs_ret_cc'] / slice_data['n_period_vol_price_terms']
        stats_data = slice_data.copy()
        stats_data.dropna(inplace=True)
        ticker_df  = pd.DataFrame(columns = stats_df.columns, index=[0])
        
        ticker_df['ticker'] = ticker
        # Now calculate the statistics for each stock and save the min a dictionary.
        
        for i in range(1, 6):
            count = (stats_data['n_period_spike'].abs() > i).sum()
            ticker_df[f'spike>{i}']  = count
        ks = stats.kstest(stats_data['log_ret_cc'],cdf='norm')
        ticker_df['ks_stat'] = ks.statistic
        ticker_df['ks_pval'] = ks.pvalue
        
        shap = stats.shapiro(stats_data['log_ret_cc'])
        ticker_df['shapiro_wilk_stat'] = shap.statistic
        ticker_df['shapiro_wilk_pval'] = shap.pvalue
        
        an = stats.anderson(stats_data['log_ret_cc'])
        ticker_df['anderson_darling_15%_pass'] = an.critical_values[0] > an.statistic
        ticker_df['anderson_darling_10%_pass'] = an.critical_values[1] > an.statistic
        ticker_df['anderson_darling_5%_pass'] = an.critical_values[2] > an.statistic
        ticker_df['anderson_darling_2.5%_pass'] = an.critical_values[3] > an.statistic
        ticker_df['anderson_darling_1%_pass'] = an.critical_values[4] > an.statistic
        ticker_df['anderson_darling_15%_pass'] = ticker_df['anderson_darling_15%_pass'].astype(bool)
        ticker_df['anderson_darling_10%_pass'] = ticker_df['anderson_darling_10%_pass'].astype(bool)
        ticker_df['anderson_darling_5%_pass'] = ticker_df['anderson_darling_5%_pass'].astype(bool)
        ticker_df['anderson_darling_2.5%_pass'] = ticker_df['anderson_darling_2.5%_pass'].astype(bool)
        ticker_df['anderson_darling_1%_pass'] = ticker_df['anderson_darling_1%_pass'].astype(bool)
        stats_df = pd.concat([ticker_df, stats_df], ignore_index=True)
else:
    select_data = dfs[TICKER]
    slice_data = select_data.loc[DATE_FROM:DATE_TO]
    slice_data.loc[:, 'lagged_close'] = slice_data['close'].shift(1)
    slice_data.loc[:, 'prior_lagged_close'] = slice_data['lagged_close'].shift(DAY_MULTIPLE)
    slice_data.loc[:, 'simple_ret_cc'] = slice_data['lagged_close'] / slice_data['prior_lagged_close']
    slice_data.loc[:, 'abs_ret_cc'] = slice_data['close'] - slice_data['close'].shift(DAY_MULTIPLE)
    slice_data.loc[:, 'log_ret_cc'] = np.log(slice_data['simple_ret_cc'])
    slice_data.loc[:, 'n_period_vol_cc'] = slice_data['log_ret_cc'].rolling(ROLLING_WINDOW).std()  # n period volatility
    slice_data.loc[:, 'annualised_vol_cc'] = slice_data['n_period_vol_cc'] * np.sqrt(DAY_MULTIPLE)
    slice_data.loc[:, 'n_period_vol_price_terms'] = slice_data['n_period_vol_cc'] * slice_data['lagged_close']
    slice_data.loc[:, 'n_period_spike'] = slice_data['abs_ret_cc'] / slice_data['n_period_vol_price_terms']
    stats_data = slice_data.copy()


In [36]:
# Now get earnings data  etc for an individual stock

earnings_df = dm.get_equity_service().get_ticker_earnings_history(TICKER, 'US')
earnings_df.set_index('reportDate', inplace=True)
earnings_slice_df = earnings_df.loc[DATE_FROM:DATE_TO]
earnings_slice_df.reset_index(inplace=True)
earnings_slice_df['adjustedReportDate'] = np.where(
    earnings_slice_df['beforeAfterMarket'] == 'AfterMarket',
    pd.to_datetime(pd.to_datetime(earnings_slice_df['reportDate'], unit='ns') + pd.Timedelta(days=1)),
    pd.to_datetime(earnings_slice_df['reportDate'], unit='ns')
)



[DEBUG]-Starting new HTTPS connection (1): eodhistoricaldata.com:443
[DEBUG]-https://eodhistoricaldata.com:443 "GET /api/fundamentals/RTX.US?fmt=json&api_token=6318e406a42036.19146604&filter=Earnings HTTP/1.1" 200 None


In [40]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=slice_data.index,
    y=slice_data['n_period_spike'],
    mode='lines',
    name='n_period_spike',
    line=dict(color='#AFBEBF', width=1)
))

# Add horizontal lines for the standard deviations
fig.add_shape(
    type='line',
    x0=slice_data.index.min(),
    x1=slice_data.index.max(),
    y0=1,
    y1=1,
    line=dict(color='lime', width=1, dash='dot')  # Fluorescent green color
)

fig.add_shape(
    type='line',
    x0=slice_data.index.min(),
    x1=slice_data.index.max(),
    y0=-1,
    y1=-1,
    line=dict(color='lime', width=1, dash='dot')  # Fluorescent green color
)
fig.add_shape(
    type='line',
    x0=slice_data.index.min(),
    x1=slice_data.index.max(),
    y0=2,
    y1=2,
    line=dict(color='pink', width=1, dash='dot')  # Fluorescent green color
)

fig.add_shape(
    type='line',
    x0=slice_data.index.min(),
    x1=slice_data.index.max(),
    y0=-2,
    y1=-2,
    line=dict(color='pink', width=1, dash='dot')  # Fluorescent green color
)
fig.add_shape(
    type='line',
    x0=slice_data.index.min(),
    x1=slice_data.index.max(),
    y0=3,
    y1=3,
    line=dict(color='red', width=1, dash='dot')  # Fluorescent green color
)

fig.add_shape(
    type='line',
    x0=slice_data.index.min(),
    x1=slice_data.index.max(),
    y0=-3,
    y1=-3,
    line=dict(color='red', width=1, dash='dot')  # Fluorescent green color
)
# Add scatter plot for earnings dates
fig.add_trace(
    go.Scatter(
        x=earnings_slice_df['reportDate'],
        y=[slice_data['n_period_spike'][date] for date in earnings_slice_df['adjustedReportDate']],
        mode='markers',
        name='Earnings Dates',
        marker=dict(color='yellow', size=6)
    )
)

fig.update_layout(showlegend=True,
                  autosize=False,
                  # Add title
                  title_text=f'{DAY_MULTIPLE} period spikes in S.D terms for ticker: {TICKER}<br>From: {DATE_FROM}<br>To: {DATE_TO}',
                  title_font=dict(size=13, color='white'),
                  # Adjust legend
                  legend=dict(
                      yanchor="top",
                      y=0.99,
                      xanchor="left",
                      x=0.01
                  ),
                  width=1200,
                  height=800,
                  plot_bgcolor='black',
                  paper_bgcolor='rgba(0,0,0,0)',  # this removes the color of the border around the plot
                  xaxis=dict(showgrid=False, gridcolor='#2A2E31', gridwidth=0.1, zeroline=False,
                             zerolinecolor='turquoise',
                             zerolinewidth=1),
                  yaxis=dict(showgrid=True, gridcolor='#2A2E31', gridwidth=0.1, zeroline=True,
                             zerolinecolor='turquoise',
                             zerolinewidth=1),
                  font=dict(color='white', size=12))

fig.show()

In [38]:
fig2 = go.Figure(data=[go.Candlestick(x=slice_data.index,
                                      open=slice_data['open'],
                                      high=slice_data['high'],
                                      low=slice_data['low'],
                                      close=slice_data['close'])])

# Copy the layout from the old chart, adjust title
fig2.update_layout(showlegend=True,
                   autosize=False,
                   title_text=f'Candlestick Chart for ticker: {TICKER}<br>From: {DATE_FROM}<br>To: {DATE_TO}',
                   title_font=dict(size=13, color='white'),
                   legend=dict(
                       yanchor="top",
                       y=0.99,
                       xanchor="left",
                       x=0.01
                   ),
                   width=1200,
                   height=600,
                   plot_bgcolor='black',
                   paper_bgcolor='rgba(0,0,0,0)',  # this removes the color of the border around the plot
                   xaxis=dict(showgrid=False, gridcolor='#2A2E31', gridwidth=0.1, zeroline=False,
                              zerolinecolor='turquoise',
                              zerolinewidth=1),
                   yaxis=dict(showgrid=True, gridcolor='#2A2E31', gridwidth=0.1, zeroline=True,
                              zerolinecolor='turquoise',
                              zerolinewidth=1),
                   font=dict(color='white', size=12))

fig2.show()